# 기본 설정

### 참고할 만한 사이트들
- Recommender using NLP: https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042  
- TfidfVectorizer 정리된 곳: https://chan-lab.tistory.com/27


In [5]:
from collections import Counter
import konlpy
from konlpy.tag import Mecab
from konlpy.tag import Okt

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import fasttext
from gensim.models import Word2Vec, FastText
from rake_nltk import Rake

In [2]:
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

### nltk가 돌아가지 않을 때 Tip!
``` python 
nltk.download()
```
- 이 코드를 돌려준다!  
<br>  

### 만약 [ SSL: certificate verify failed ] 에러가 뜬다면?!?!
- python 3.8 폴더로 들어간다!
- install Certificates.command를 실행시켜준다!
- 그 다음에 nltk.download()를 다시 실행시켜주면 된다!!


In [2]:
import nltk

In [8]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [9]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/junghyunwoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 시자아아악~~

In [14]:
df = pd.read_csv('movieData.csv')

df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [15]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [16]:
df

,Title,Genre,Director,Actors,Key_words
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","[years, eventual, redemption, two, imprisoned,..."
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[organized, crime, dynasty, transfers, control..."
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...","[tightens, grip, 1920s, new, york, son, michae..."
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","[mysterious, past, people, ability, wreaks, ha..."
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....","[colleagues, forcing, justice, evidence, recon..."
...,...,...,...,...,...
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...","[four, followed, day, drinking, bout, desperat..."
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...","[world, alongside, something, supervising, sta..."
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...","[wife, book, newspaper, editor, uses, every, t..."
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...","[lawn, long, journey, mend, mover, tractor, il..."


In [19]:
# instantiating and generating the count matrix
count = CountVectorizer()

In [21]:
count_matrix = count.fit_transform(df[['Title', 'Genre', 'Director', 'Actors', 'Key_words']])

In [22]:
count_matrix

<5x5 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [23]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [25]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [28]:
recommendations('Fargo', cosine_sim=cosine_sim)

IndexError: index 0 is out of bounds for axis 0 with size 0

# NLP로 recommend system 만들기!
https://www.kdnuggets.com/2019/11/content-based-recommender-using-natural-language-processing-nlp.html

In [93]:
from rake_nltk import Rake
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('movieData.csv')
del df['Unnamed: 0']
df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


## 줄거리 핵심만 남기기

In [94]:
r = Rake()
def rake_column(column):
    r.extract_keywords_from_text(column)
    key_words_dict_scores = r.get_word_degrees()
    return list( key_words_dict_scores.keys())

In [95]:
df['Key_words'] = df['Plot'].apply(rake_column)

In [96]:
df[['Key_words', 'Plot']]

,Key_words,Plot
0,"[two, imprisoned, men, bond, years, acts, find...",Two imprisoned men bond over a number of years...
1,"[clandestine, empire, aging, patriarch, organi...",The aging patriarch of an organized crime dyna...
2,"[grip, michael, career, tightens, family, crim...",The early life and career of Vito Corleone in ...
3,"[people, greatest, psychological, dark, knight...",When the menace known as the Joker emerges fro...
4,"[evidence, colleagues, jury, holdout, attempts...",A jury holdout attempts to prevent a miscarria...
...,...,...
245,"[followed, desperate, life, day, drinking, bou...",The desperate life of a chronic alcoholic is f...
246,"[troubled, waters, worker, longtime, boyfriend...",A 20-something supervising staff member of a r...
247,"[keep, ace, reporter, ex, newspaper, editor, u...",A newspaper editor uses every trick in the boo...
248,"[mover, tractor, old, man, makes, long, journe...",An old man makes a long journey by lawn-mover ...


In [97]:
df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response,Key_words
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True,"[two, imprisoned, men, bond, years, acts, find..."
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True,"[clandestine, empire, aging, patriarch, organi..."
2,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True,"[grip, michael, career, tightens, family, crim..."
3,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True,"[people, greatest, psychological, dark, knight..."
4,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True,"[evidence, colleagues, jury, holdout, attempts..."


## 다른 column들 전처리해주기

In [131]:
def lower_sentences(column):
    content = column.lower().replace(' ','')
    content = content.replace(',', ' ')
    return content

In [132]:
temp = df.copy()

In [133]:
temp['Genre'] = temp['Genre'].apply(lower_sentences)
temp['Actors'] = temp['Actors'].apply(lower_sentences)
temp['Director'] = temp['Director'].apply(lower_sentences)

In [134]:
temp.head(2)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response,Key_words
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,crime drama,frankdarabont,"Stephen King (short story ""Rita Hayworth and S...",timrobbins morganfreeman bobgunton williamsadler,Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True,"[two, imprisoned, men, bond, years, acts, find..."
1,The Godfather,1972,R,24 Mar 1972,175 min,crime drama,francisfordcoppola,"Mario Puzo (screenplay), Francis Ford Coppola ...",marlonbrando alpacino jamescaan richards.caste...,The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True,"[clandestine, empire, aging, patriarch, organi..."


In [136]:
def concat_columns(columns):
    content = ''
    for col in columns:
        content = content + col
        
    return content

In [135]:
temp['Genre'][0]

'crime drama'

In [144]:
temp['Bag_of_words'] = (temp['Genre'].astype(str) + ' ' + temp['Director'].astype(str) + ' ' +  temp['Actors'].astype(str) + ' ' + temp['Key_words'].astype(str))

In [145]:
temp

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response,Key_words,Bag_of_words
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,crime drama,frankdarabont,"Stephen King (short story ""Rita Hayworth and S...",timrobbins morganfreeman bobgunton williamsadler,Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True,"[two, imprisoned, men, bond, years, acts, find...",crime drama frankdarabont timrobbins morganfre...
1,The Godfather,1972,R,24 Mar 1972,175 min,crime drama,francisfordcoppola,"Mario Puzo (screenplay), Francis Ford Coppola ...",marlonbrando alpacino jamescaan richards.caste...,The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True,"[clandestine, empire, aging, patriarch, organi...",crime drama francisfordcoppola marlonbrando al...
2,The Godfather: Part II,1974,R,20 Dec 1974,202 min,crime drama,francisfordcoppola,"Francis Ford Coppola (screenplay), Mario Puzo ...",alpacino robertduvall dianekeaton robertdeniro,The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True,"[grip, michael, career, tightens, family, crim...",crime drama francisfordcoppola alpacino robert...
3,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,action crime drama,christophernolan,"Jonathan Nolan (screenplay), Christopher Nolan...",christianbale heathledger aaroneckhart michael...,When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True,"[people, greatest, psychological, dark, knight...",action crime drama christophernolan christianb...
4,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,crime drama,sidneylumet,"Reginald Rose (story), Reginald Rose (screenplay)",martinbalsam johnfiedler leej.cobb e.g.marshall,A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True,"[evidence, colleagues, jury, holdout, attempts...",crime drama sidneylumet martinbalsam johnfiedl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Lo

In [146]:
df = temp[['Title','Bag_of_words']]

In [148]:
df

,Title,Bag_of_words
0,The Shawshank Redemption,crime drama frankdarabont timrobbins morganfre...
1,The Godfather,crime drama francisfordcoppola marlonbrando al...
2,The Godfather: Part II,crime drama francisfordcoppola alpacino robert...
3,The Dark Knight,action crime drama christophernolan christianb...
4,12 Angry Men,crime drama sidneylumet martinbalsam johnfiedl...
...,...,...
245,The Lost Weekend,drama film-noir billywilder raymilland janewym...
246,Short Term 12,drama destindanielcretton brielarson johngalla...
247,His Girl Friday,comedy drama romance howardhawks carygrant ros...
248,The Straight Story,biography drama davidlynch sissyspacek janegal...


In [156]:
tfid = TfidfVectorizer()
tfid_matrix = tfid.fit_transform(df['Bag_of_words'])
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)
print(cosine_sim)

[[1.         0.02693012 0.02491061 ... 0.0036137  0.00358765 0.00349224]
 [0.02693012 1.         0.17513483 ... 0.00377687 0.00374964 0.00364992]
 [0.02491061 0.17513483 1.         ... 0.00349364 0.00346845 0.00337621]
 ...
 [0.0036137  0.00377687 0.00349364 ... 1.         0.00365226 0.00355513]
 [0.00358765 0.00374964 0.00346845 ... 0.00365226 1.         0.0035295 ]
 [0.00349224 0.00364992 0.00337621 ... 0.00355513 0.0035295  1.        ]]


In [165]:
sorted(tfid.vocabulary_.items())

[('000', 0),
 ('10', 1),
 ('100', 2),
 ('12', 3),
 ('17', 4),
 ('1820s', 5),
 ('18th', 6),
 ('1900', 7),
 ('1920s', 8),
 ('1936', 9),
 ('1940s', 10),
 ('1941', 11),
 ('1948', 12),
 ('1950s', 13),
 ('1951', 14),
 ('1954', 15),
 ('1960s', 16),
 ('1970s', 17),
 ('1972', 18),
 ('1979', 19),
 ('1980', 20),
 ('1984', 21),
 ('19th', 22),
 ('20', 23),
 ('2029', 24),
 ('24', 25),
 ('25', 26),
 ('30', 27),
 ('40', 28),
 ('7th', 29),
 ('9000', 30),
 ('aaroneckhart', 31),
 ('abagnale', 32),
 ('abandoned', 33),
 ('abandons', 34),
 ('abducted', 35),
 ('abhishekbharate', 36),
 ('ability', 37),
 ('able', 38),
 ('abrams', 39),
 ('accelerated', 40),
 ('accept', 41),
 ('accepted', 42),
 ('accepts', 43),
 ('accident', 44),
 ('accidentally', 45),
 ('according', 46),
 ('account', 47),
 ('accumulates', 48),
 ('accuracy', 49),
 ('accused', 50),
 ('accuses', 51),
 ('ace', 52),
 ('achieve', 53),
 ('acrophobia', 54),
 ('across', 55),
 ('action', 56),
 ('activities', 57),
 ('actress', 58),
 ('acts', 59),
 ('actua

In [163]:
tfid_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [157]:
indices = pd.Series(df['Title'])

In [158]:
def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_movies.append(list(df['Title'])[i])
        
    return recommended_movies

In [159]:
indices[:50]

0                              The Shawshank Redemption
1                                         The Godfather
2                                The Godfather: Part II
3                                       The Dark Knight
4                                          12 Angry Men
5                                      Schindler's List
6         The Lord of the Rings: The Return of the King
7                                          Pulp Fiction
8                                            Fight Club
9     The Lord of the Rings: The Fellowship of the Ring
10                                         Forrest Gump
11       Star Wars: Episode V - The Empire Strikes Back
12                                            Inception
13                The Lord of the Rings: The Two Towers
14                      One Flew Over the Cuckoo's Nest
15                                           Goodfellas
16                                           The Matrix
17                   Star Wars: Episode IV - A N

In [160]:
recommend('The Lion King')

['Monsters, Inc.',
 'WALL·E',
 'The Nightmare Before Christmas',
 "One Flew Over the Cuckoo's Nest",
 'Butch Cassidy and the Sundance Kid',
 'Monty Python and the Holy Grail',
 'Die Hard',
 'Song of the Sea',
 'V for Vendetta',
 'Braveheart']

In [152]:
recommend('The Avengers')

['Guardians of the Galaxy Vol. 2',
 'Aliens',
 'Guardians of the Galaxy',
 'The Martian',
 'Interstellar',
 'Blade Runner',
 'Terminator 2: Judgment Day',
 'The Thing',
 'The Terminator',
 'Spider-Man: Homecoming']

In [154]:
recommend('The Godfather')

['The Godfather: Part II',
 'Scarface',
 'Fargo',
 'Rope',
 'On the Waterfront',
 'Goodfellas',
 'Baby Driver',
 'Cool Hand Luke',
 'Casino',
 'A Clockwork Orange']

# petr.eat 데이터로!

In [36]:
df = pd.read_excel('제품별 원재료 및 성분표.xlsx')

In [37]:
temp = df.copy()

In [38]:
temp.head(2)

,상품명,제조사,딱딱한 정도,주요성분,영양소,어디에 좋냐,효능,특징
0,한우사골육수,바르다펫,액체,"우족, 사골뼈, 황태, 표고버섯,대추","콜라겐, 콘드로이틴","피부건강, 관절",기력회복,기호성 좋아 특식과 함께 곁들여 먹여라
1,견냥갱(장어),바르다펫,푸딩,"바다장어, 황태, 한천가루","불포화지방산, 오메가3, 철분과 칼슘 비타민A, 필수지방산 DHA, EPA, 바다장...","혈액순환, 성장발육, 뇌건강, 슬개골탈구예방, 관절건강",NaN,NaN


In [39]:
temp.columns

Index(['상품명', '제조사', '딱딱한 정도', '주요성분 ', '영양소 ', '어디에 좋냐', '효능', '특징'], dtype='object')

In [40]:
temp['주요성분 '] = temp['주요성분 '].str.replace(',', ' ')
temp['영양소 '] = temp['영양소 '].str.replace(',', ' ')
temp['어디에 좋냐'] = temp['어디에 좋냐'].str.replace(',', ' ')
temp['효능'] = temp['효능'].str.replace(',', ' ')

In [50]:
temp = temp.fillna('')

In [51]:
temp['BOW'] = (temp['주요성분 '].astype(str) + ' ' 
               + temp['영양소 '].astype(str) + ' ' 
               + temp['어디에 좋냐'].astype(str) + ' ' 
               + temp['효능'].astype(str))

In [52]:
temp

,상품명,제조사,딱딱한 정도,주요성분,영양소,어디에 좋냐,효능,특징,BOW
0,한우사골육수,바르다펫,액체,우족 사골뼈 황태 표고버섯 대추,콜라겐 콘드로이틴,피부건강 관절,기력회복,기호성 좋아 특식과 함께 곁들여 먹여라,우족 사골뼈 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절 기력회복
1,견냥갱(장어),바르다펫,푸딩,바다장어 황태 한천가루,불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방산 DHA EPA 바다장...,혈액순환 성장발육 뇌건강 슬개골탈구예방 관절건강,,,바다장어 황태 한천가루 불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방...
2,치킨사골육수,바르다펫,액체,닭발 닭가슴살 황태 표고버섯 대추,콜라겐 콘드로이틴,피부건강 관절건강,기력회복,,닭발 닭가슴살 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절건강 ...
3,스팀제주말고기소시지,바르다펫,부드러움,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈,팔미톨레산 베타카로틴 베타카로틴 칼슘 비타민 안토시아닌 비타민 무기질 ...,췌장 인슐린 당뇨 항산화작용 활성산소배출 노화방지 눈건강 노화방지 ...,,,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈 팔미톨레산 베타...
4,강아지 보양식 바른 삼계탕,바르다펫,부드러움,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...,콜라겐 콘드로이틴,피부 관절건강,,,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...
5,한입오리,바르다펫,촉촉한 육포,오리안심,단백질 불포화지방산,피부 혈관건강,,,오리안심 단백질 불포화지방산 피부 혈관건강
6,한입치킨,바르다펫,촉촉한 육포,닭안심,단백질 니아신,근육강화 신진대사 면역력,,,닭안심 단백질 니아신 근육강화 신진대사 면역력
7,무염황태체,바르다펫,바삭바삭,황태,아미노산 메티오닌,간기능,,,황태 아미노산 메티오닌 간기능
8,오리목뼈,바르다펫,딱딱,오리목뼈,단백질 무기질,콜레스테롤저하 독소배출,스트레스해소 치석제거,,오리목뼈 단백질 무기질 콜레스테롤저하 독소배출 스트레스해소 치석제거
9,홍연어저키,바르다펫,마른육포,홍연어,DHA EPA 오메가3 저지방산,향산화효과 시력개선 심혈관개선 면역력강화 항암효과,,,홍연어 DHA EPA 오메가3 저지방산 향산화효과 시력개선 심혈관개선 ...


In [53]:
df = temp[['상품명', 'BOW']]

In [54]:
df

,상품명,BOW
0,한우사골육수,우족 사골뼈 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절 기력회복
1,견냥갱(장어),바다장어 황태 한천가루 불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방...
2,치킨사골육수,닭발 닭가슴살 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절건강 ...
3,스팀제주말고기소시지,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈 팔미톨레산 베타...
4,강아지 보양식 바른 삼계탕,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...
5,한입오리,오리안심 단백질 불포화지방산 피부 혈관건강
6,한입치킨,닭안심 단백질 니아신 근육강화 신진대사 면역력
7,무염황태체,황태 아미노산 메티오닌 간기능
8,오리목뼈,오리목뼈 단백질 무기질 콜레스테롤저하 독소배출 스트레스해소 치석제거
9,홍연어저키,홍연어 DHA EPA 오메가3 저지방산 향산화효과 시력개선 심혈관개선 ...


In [55]:
tfid = TfidfVectorizer()
tfid_matrix = tfid.fit_transform(df['BOW'])
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)
print(cosine_sim)

[[1.         0.10046301 0.68658542 ... 0.         0.         0.        ]
 [0.10046301 1.         0.16439049 ... 0.         0.         0.        ]
 [0.68658542 0.16439049 1.         ... 0.         0.09663947 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.09663947 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [56]:
indices = pd.Series(df['상품명'])

In [72]:
def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_movies.append(list(df['상품명'])[i])
        
    return recommended_movies

In [73]:
indices[:20]

0             한우사골육수
1            견냥갱(장어)
2             치킨사골육수
3         스팀제주말고기소시지
4     강아지 보양식 바른 삼계탕
5               한입오리
6               한입치킨
7             무염황태체 
8               오리목뼈
9              홍연어저키
10           제주말고기육포
11          견냥갱(소고기)
12            치킨안심육포
13          스팀 오리소시지
14            바르다 화식
15    단호박 연어 파운드 케이크
16             송아지목뼈
17            오리안심육포
18          시금치케일우유껌
19         블루베리비트우유꼄
Name: 상품명, dtype: object

In [74]:
df.head(15)

,상품명,BOW
0,한우사골육수,우족 사골뼈 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절 기력회복
1,견냥갱(장어),바다장어 황태 한천가루 불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방...
2,치킨사골육수,닭발 닭가슴살 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절건강 ...
3,스팀제주말고기소시지,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈 팔미톨레산 베타...
4,강아지 보양식 바른 삼계탕,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...
5,한입오리,오리안심 단백질 불포화지방산 피부 혈관건강
6,한입치킨,닭안심 단백질 니아신 근육강화 신진대사 면역력
7,무염황태체,황태 아미노산 메티오닌 간기능
8,오리목뼈,오리목뼈 단백질 무기질 콜레스테롤저하 독소배출 스트레스해소 치석제거
9,홍연어저키,홍연어 DHA EPA 오메가3 저지방산 향산화효과 시력개선 심혈관개선 ...


In [75]:
recommend('한입오리')

['오리안심육포',
 '오리황태말이',
 '오리안심육포',
 '스팀 오리소시지',
 '스팀 치킨소시지',
 '오리장각',
 '소떡심',
 '코코넛먼치킨 ',
 '한입치킨',
 '치킨안심육포']

In [77]:
recommend('치킨안심육포')

['한입치킨',
 '스팀 치킨소시지',
 '오리황태말이',
 '캥거루미트볼',
 '코코넛먼치킨 ',
 '소떡심',
 '야채듬뿍테린 ',
 '견냥갱(치킨)',
 '말고기 치즈밀 ',
 '오리안심육포']